In [52]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import json
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [53]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

In [54]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 归还逾期

In [55]:
# 归还逾期overdue_type=1，2为租赁期间有逾期
sql = '''
select
yat.order_id, yat.add_time, yat.allot_time, yat.agent_user_name,yat.max_day, tprm.search_time, om.status, om.has_actual, tprm.relet_days, tprm.relet_periods
from (select distinct order_id,add_time,agent_user_name,allot_time, max_day from db_rent.ya_agent_task where overdue_type=1) yat
left join db_digua_business.t_order om on om.id=yat.order_id
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=yat.order_id
-- where  yat.max_day IN (26,27,29,31) 
'''
df = query(sql)
# df.drop_duplicates(subset=['order_id'], inplace=True)

In [56]:
t_date = datetime.now().strftime('%Y%m%d%H')
# allot_time任务分配时间
df.loc[:, 'date'] = df.allot_time.dt.strftime('%Y-%m-%d')
df.loc[:, '姓名'] = df.agent_user_name.apply(lambda x: str(x).split('-')[-1])
df.loc[:, '已买断'] = np.where((df.status==8)&(df.has_actual==1)&(df.relet_days==0), 1, 0)
df.loc[:, '已归还'] = np.where((df.status.isin([8, 15, 16]))&(df.has_actual==0)&(df.relet_days==0), 1, 0)
df.loc[:, '已续租'] = np.where(df.relet_days>0, 1, 0)
df.loc[:, '仍在逾期'] = np.where((df.status==5)&(df.relet_days==0), 1, 0)
df_g = df.groupby('search_time').agg({'order_id': 'count', '已买断': 'sum', '已归还': 'sum', '已续租': 'sum', '仍在逾期': 'sum'}).rename(columns={'order_id': '归还逾期数量'})
df_g = df_g.rename_axis('下单月份')

# # df_g.to_excel('F:/x.xlsx')
with pd.ExcelWriter(f'F:/需求/仕程需求/归还逾期_{t_date}.xlsx', engine='xlsxwriter') as writer:
    df_g.to_excel(writer, sheet_name='归还逾期')

In [57]:
df_g


,归还逾期数量,已买断,已归还,已续租,仍在逾期
下单月份,,,,,
2022-07,22,4,0,18,0
2022-08,100,12,9,36,43
2022-09,85,1,1,24,59
2022-10,36,7,3,15,11
2022-11,13,0,1,1,11
2022-12,40,10,0,0,30
2023-01,130,10,8,1,111
2023-02,212,20,15,0,177
2023-03,112,15,2,1,94


## 催收完成情况

In [7]:

df_new = df[(df.date>='2025-06-01')&(df.date<='2025-06-30')&(df.姓名.isin(['吴威宇','陈锐', '张文奔', '郑瑞杰']))]
df_new = df_new.sort_values('allot_time').groupby('order_id').head(1)
df_new_g = df_new.groupby('姓名').agg({'order_id': 'count', '已买断': 'sum', '已归还': 'sum', '已续租': 'sum', '仍在逾期': 'sum'}).rename(columns={'order_id': '分配数'})
df_new_g.loc[:, '总完成'] = df_new_g.已买断+df_new_g.已归还+df_new_g.已续租
df_new_g.loc[:, '总完成率'] = (df_new_g.总完成/df_new_g.分配数).map(lambda x: format(x, '.2%'))
# df_new_g = df_new_g[['分配数', '总完成', '总完成率']]
with pd.ExcelWriter('F:/需求/仕程需求/完成情况_6月.xlsx', engine='xlsxwriter') as writer:
    df_new_g.to_excel(writer, sheet_name='完成率')

# 租金逾期

In [37]:
# tprm.overdue_type,yat.催收状态,3为已完成其它都为逾期
sql_zj = '''
select
yat.order_id, yat.add_time, yat.allot_time, yat.催收状态, yat.agent_user_name, yat.max_day, tprm.search_time, om.status, om.has_actual, tprm.relet_days, tprm.relet_periods
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2
,ymos.refund_date, ymos.reality_refund_date
from (select distinct order_id,add_time,agent_user_name,allot_time,status 催收状态, max_day from db_rent.ya_agent_task where overdue_type=0) yat
left join db_digua_business.t_order om on om.id=yat.order_id
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=yat.order_id
left join db_rent.ya_merchant_order_stages ymos on ymos.order_id=om.id
where date_format(add_time,'%Y-%m')='2025-06'
and  yat.max_day IN (26,27,29,31) 
'''
df_zj = query(sql_zj)

In [38]:
df_zj#.agent_user_name.unique()

,order_id,add_time,allot_time,催收状态,agent_user_name,max_day,search_time,status,has_actual,relet_days,relet_periods,status2,refund_date,reality_refund_date
0,428973,2025-06-06 08:30:02,2025-06-06 09:01:13,3,自有催收组-张文奔,31,2023-06,8,1,360,12,已完成,2023-07-03,2023-06-30 11:10:24
1,428973,2025-06-06 08:30:02,2025-06-06 09:01:13,3,自有催收组-张文奔,31,2023-06,8,1,360,12,已完成,2023-08-03,2023-07-30 09:36:00
2,428973,2025-06-06 08:30:02,2025-06-06 09:01:13,3,自有催收组-张文奔,31,2023-06,8,1,360,12,已完成,2023-09-03,2023-08-30 14:19:06
3,428973,2025-06-06 08:30:02,2025-06-06 09:01:13,3,自有催收组-张文奔,31,2023-06,8,1,360,12,已完成,2023-10-03,2023-09-29 22:12:42
4,428973,2025-06-06 08:30:02,2025-06-06 09:01:13,3,自有催收组-张文奔,31,2023-06,8,1,360,12,已完成,2023-11-03,2023-11-03 11:16:52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36480,2718376,2025-06-30 08:30:03,2025-06-30 09:03:03,5,自有催收组-吴威宇,31,2025-05,4,0,0,0,租赁中,2025-12-26,NaT
36481,2718376,2025-06-30 08:30:03,2025-06-30 09:03:03,5,自有催收组-吴威宇,31,2025-05,4,0,0,0,租赁中,2026-01-26,NaT
36482,2718376,2025-06-30 08:30:03,2025-06-30 09:03:03,5,自有催收组-吴威宇,31,2025-05,4,0,0,0,租赁中,2026-02-26,NaT
36483,2718376,2025-06-30 08:30:03,2025-06-30 09:03:03,5,自有催收组-吴威宇,31,2025-05,4,0,0,0,租赁中,2026-03-26,NaT


In [50]:
t_date = datetime.now().strftime('%Y%m%d%H')
name_list = ['自有催收组-吴威宇','自有催收组-陈锐', '自有催收组-张文奔', '自有催收组-郑瑞杰']
# 筛选agent_user_name包含深圳市润广贸易有限公司和在name_list中的人
df_new = df_zj[df_zj.agent_user_name.str.contains('深圳市润广贸易有限公司') | df_zj.agent_user_name.isin(name_list)]
# allot_time任务分配时间
df_new.loc[:, 'date'] = df_new.allot_time.dt.strftime('%Y-%m-%d')
df_new.loc[:, '姓名'] = df_new.agent_user_name
df_new.loc[:, '已买断'] = np.where((df_new.status.isin([3, 4]))&(df_new.has_actual==1), 1, 0)
df_new.loc[:, '已还款'] = np.where((df_new.催收状态.isin([3, 4])), 1, 0)
# df_new.loc[:, '已续租'] = np.where(df_new.relet_days>0, 1, 0)
df_new.loc[:, '仍在逾期'] = np.where(~df_new.催收状态.isin([3, 4]), 1, 0)


# df_new = df[(df.date>='2025-06-01')&(df.date<='2025-06-30')&(df.姓名.isin(['吴威宇','陈锐', '张文奔', '郑瑞杰']))]
df_new.drop_duplicates(subset=['order_id'], inplace=True)
df_new = df_new.sort_values('allot_time').groupby('order_id').head(1)
df_new_g = df_new.groupby('姓名').agg({'order_id': 'count', '已买断': 'sum', '已还款': 'sum','仍在逾期': 'sum'}).rename(columns={'order_id': '分配数'})
df_new_g.loc[:, '总完成'] = df_new_g.已买断+df_new_g.已还款
df_new_g.loc[:, '总完成率'] = (df_new_g.总完成/df_new_g.分配数).map(lambda x: format(x, '.2%'))

with pd.ExcelWriter('F:/需求/仕程需求/租金催收完成情况_6月.xlsx', engine='xlsxwriter') as writer:
    df_new_g.to_excel(writer, sheet_name='完成率')
df_new_g

,分配数,已买断,已还款,仍在逾期,总完成,总完成率
姓名,,,,,,
深圳市润广贸易有限公司-chenzemei,121,0,64,57,64,52.89%
深圳市润广贸易有限公司-mocaidie,127,0,68,59,68,53.54%
深圳市润广贸易有限公司-wucheng,123,0,74,49,74,60.16%
深圳市润广贸易有限公司-wuwen003,105,0,64,41,64,60.95%
深圳市润广贸易有限公司-ying,175,0,98,77,98,56.00%
深圳市润广贸易有限公司-zhuxiaojuan,168,0,97,71,97,57.74%
自有催收组-吴威宇,449,0,288,161,288,64.14%
自有催收组-张文奔,449,0,329,120,329,73.27%
自有催收组-郑瑞杰,450,0,327,123,327,72.67%


In [7]:
sql2 = '''
select
yat.order_id, yat.add_time, yat.sort, tprm.search_time, om.status, om.has_actual, tprm.relet_days, tprm.relet_periods
from (select order_id, add_time, sort from db_rent.ya_agent_task where overdue_type=0) yat
left join db_digua_business.t_order om on om.id=yat.order_id
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=yat.order_id
'''
df2 = query(sql2)
df2.drop_duplicates(subset=['order_id'], inplace=True)

In [8]:
df2.loc[:, 'add_date'] = pd.to_datetime(df2.add_time.dt.date)
df2_new = df2[(df2.add_date>=pd.to_datetime('2024-10-01'))&(df2.add_date<=pd.to_datetime('2025-03-31'))]
df2_new

,order_id,add_time,sort,search_time,status,has_actual,relet_days,relet_periods,add_date
21278,351928,2024-10-01 00:30:02,[17],2023-05,8,1,184.0,7.0,2024-10-01
21280,424930,2024-10-01 00:30:02,[16],2023-06,4,0,360.0,12.0,2024-10-01
21282,425376,2024-10-01 00:30:02,[16],2023-06,4,0,122.0,5.0,2024-10-01
21283,425924,2024-10-01 00:30:02,[16],2023-06,8,1,332.0,12.0,2024-10-01
21286,500879,2024-10-01 00:30:02,[15],2023-07,4,0,62.0,3.0,2024-10-01
...,...,...,...,...,...,...,...,...,...
58986,1926032,2025-03-31 00:30:04,[4],2024-12,4,0,0.0,0.0,2025-03-31
58987,1926180,2025-03-31 00:30:04,[4],2024-12,4,0,0.0,0.0,2025-03-31
58988,1926626,2025-03-31 00:30:04,[4],2024-12,4,0,0.0,0.0,2025-03-31
58991,2048920,2025-03-31 00:30:04,[3],2025-01,4,0,0.0,0.0,2025-03-31
